In [ ]:
!git clone https://github.com/DanielK345/Time-Series-Library
%cd Time-Series-Library

In [ ]:
!nvidia-smi

In [1]:
!nvidia-smi

In [ ]:
print("Hello World")

In [2]:
%ls

In [ ]:
libs = \
"einops==0.8.0 \
local-attention==1.9.14 \
patool==1.12 \
reformer-pytorch==1.4.4 \
sktime==0.16.1 \
sympy==1.11.1 \
tqdm==4.64.1 \
PyWavelets \
numpy==1.24.4 \
pandas==1.5.3 \
scikit-learn==1.2.2 \
scipy==1.10.1 \
dask[dataframe] \
axial-positional-embedding"
!pip install --upgrade --force-reinstall {libs}

In [ ]:
from google.colab import drive
import os
import numpy as np
import pandas as pd
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import torch
import scipy
import einops
import sktime
import local_attention
import patoolib
import argparse
#import reformer_pytorch
# einops==0.8.0
# local-attention==1.9.14
# patool==1.12
# reformer-pytorch==1.4.4
# scikit-learn==1.2.2
# scipy==1.10.1
# sktime==0.16.1
# sympy==1.11.1
# tqdm==4.64.1
# PyWavelets

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Set the path to your folder in Drive
root_path = '/content/drive/MyDrive/TS'  # change this!

In [ ]:
csv_files_list = []
for dirpath, _, filenames in os.walk(root_path):
    for f in filenames:
        if f.endswith('.csv'):
            csv_files_list.append(os.path.join(dirpath, f))
csv_files_list

In [ ]:
etth1_path = csv_files_list[4]
df_etth1 = pd.read_csv(etth1_path)
#df_etth1['date'] = pd.to_datetime(df_etth1['date'])
df_etth1.head(20)

In [ ]:
sys.path.append('/content/Time-Series-Library')

In [ ]:
from data_provider.data_loader import Dataset_ETT_hour, Dataset_ETT_minute
from exp.exp_long_term_forecasting import Exp_Long_Term_Forecast

In [ ]:
train_data = Dataset_ETT_hour(
    args,
    root_path=root_path,
    flag='train',
    features='M',
    data_path='ETTh1.csv',
    timeenc=1
)

x, y, x_mark, y_mark = train_data[0]

In [ ]:
x.shape, y.shape, x_mark.shape, y_mark.shape

In [ ]:
args_dict = {
    'model': 'Autoformer',
    'model_id': 'ETTh1_96_96',
    'data': 'ETTh1',
    'root_path': root_path,
    'data_path': 'ETTh1.csv',
    'features': 'M',           # 'M' for multivariate, 'S' for univariate, 'MS' for multi-to-single
    'seq_len': 96,
    'label_len': 48,
    'pred_len': 96,
    'e_layers': 2,
    'd_layers': 1,
    'factor': 3,
    'enc_in': 7,
    'dec_in': 7,
    'c_out': 7,
    'des': 'Exp',
    'itr': 1,
    'batch_size': 32,
    'learning_rate': 0.001,
    'train_epochs': 20,
    'patience': 7,
    'use_gpu': True,
    'use_multi_gpu': False,
    'device_ids': [0],
    'gpu_type': 'cuda',
    'use_amp': False,
    'checkpoints': './checkpoints/',
    'embed': 'timeF',
    'inverse': False,
    'num_workers': 2,
    'use_dtw': False,
    'augmentation_ratio': 0,
    'task_name': 'long_term_forecast', # Added task_name
    'moving_avg': 5, # Added moving_avg
    'distil': True, # Added distil
    'is_training': 1 # Changed from True to 1
}

args = argparse.Namespace(**args_dict)
print(vars(args))

In [ ]:
# Convert args_dict to a string of command-line arguments
args_list = []
for key, value in args_dict.items():
    if key == 'use_gpu':
        # Handle use_gpu specifically as it expects a value
        args_list.append(f'--{key}')
        args_list.append(str(value))
    elif isinstance(value, bool):
        if value:
            args_list.append(f'--{key}')
    elif key == 'device_ids' and not args_dict.get('use_multi_gpu', False):
        # Omit device_ids if use_multi_gpu is False
        continue
    elif isinstance(value, list):
        args_list.append(f'--{key}')
        args_list.extend([str(item) for item in value])
    else:
        args_list.append(f'--{key}')
        args_list.append(str(value))

args_string = ' '.join(args_list)

# Construct the full command
command = f"python run.py {args_string}"

# Execute the command
!{command}

In [ ]:
#Load model form checkpoints
configs = {
    'model': 'Autoformer',
    'model_id': 'ETTh1_96_96',
    'data': 'ETTh1',
    'root_path': root_path,
    'data_path': 'ETTh1.csv',
    'features': 'M',           # 'M' for multivariate, 'S' for univariate, 'MS' for multi-to-single
    'seq_len': 96,
    'label_len': 48,
    'pred_len': 96,
    'e_layers': 2,
    'd_layers': 1,
    'factor': 3,
    'enc_in': 7,
    'dec_in': 7,
    'c_out': 7,
    'des': 'Exp',
    'itr': 1,
    'batch_size': 32,
    'learning_rate': 0.001,
    'train_epochs': 20,
    'patience': 5,
    'use_gpu': True,
    'use_multi_gpu': False,
    'device_ids': [0],
    'gpu_type': 'cuda',
    'use_amp': False,
    'checkpoints': './checkpoints/',
    'inverse': False,
    'num_workers': 2,
    'use_dtw': False,
    'augmentation_ratio': 0,
    'task_name': 'long_term_forecast', # Added task_name
    'moving_avg': 5, # Corrected moving_avg
    'distil': True, # Added distil
    'd_model': 512, # Added d_model
    'n_heads': 8, # Added n_heads
    'dropout': 0.1, # Added dropout
    'embed': 'timeF', # Added embed
    'activation': 'gelu', # Added activation
    'output_attention': False, # Added output_attention
    'do_predict': False, # Added do_predict
    'freq': 'h', # Added freq
    'd_ff': 2048 # Added d_ff
}
configs = argparse.Namespace(**configs)
from models.Autoformer import Model as Autoformer
model = Autoformer(configs)
#checkpoint_path = '/content/Time-Series-Library/checkpoints/long_term_forecast_ETTh1_96_96_Autoformer_ETTh1_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtFalse_Exp_0/checkpoint.pth'
#Upload checkpoint
checkpoint_path = '/content/checkpoint.pth'
model.load_state_dict(torch.load(checkpoint_path))
model.eval()

In [ ]:
results_path = '/content/Time-Series-Library/results'
npy_files_list = []
for dirpath, _, filenames in os.walk(results_path):
    for f in filenames:
        if f.endswith('.npy'):
            npy_files_list.append(os.path.join(dirpath, f))
npy_files_list

In [ ]:
metrics = np.load(npy_files_list[0])
metrics

In [ ]:
pred, true = np.load(npy_files_list[1]), np.load(npy_files_list[2])

In [ ]:
%matplotlib inline

In [ ]:
pred.shape

In [ ]:
batch = 0
time_f = 5
output_feature0 = pred[batch, :, time_f]
truth_feature0 = true[batch, :, time_f]
plt.figure(figsize=(20, 6))
plt.plot(output_feature0, label='pred')
plt.plot(truth_feature0, label='true')
plt.legend()
plt.show()

In [ ]:
batch = 1
output_feature0 = pred[batch, :, 0]
truth_feature0 = true[batch, :, 0]
plt.figure(figsize=(20, 6))
plt.plot(output_feature0, label='pred')
plt.plot(truth_feature0, label='true')
plt.legend()
plt.show()

In [ ]:
len(df_etth1)

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler # Import StandardScaler
from utils.timefeatures import (
    SecondOfMinute, MinuteOfHour, HourOfDay, DayOfWeek,
    DayOfMonth, DayOfYear, MonthOfYear
)

def time_features(dt_index):
    """
    Returns 4 time features for each timestamp in dt_index to match model input.
    Output shape: (len(dt_index), 4)
    """
    features = [
        HourOfDay(),
        DayOfWeek(),
        DayOfMonth(),
        DayOfYear(),
    ]
    # Ensure dt_index is a pandas DatetimeIndex
    if not isinstance(dt_index, pd.DatetimeIndex):
      dt_index = pd.to_datetime(dt_index)

    feats = np.stack([feat(dt_index) for feat in features], axis=1)
    return feats


def prepare_eval_sample(
    series,
    timestamps, # Add timestamps as an argument
    seq_len=96,
    label_len=48,
    pred_len=96,
    freq='h',
    features='M',  # 'S', 'M', or 'MS'
    target_index=0  # used only for 'S' and 'MS'
):
    """
    Prepares a single evaluation sample for the model from a time series.
    Returns: batch_x, batch_y, batch_x_mark, batch_y_mark, dec_inp, scaler
    """
    # Ensure data is a numpy array of float type
    series = np.array(series, dtype=np.float32)
    L = seq_len + pred_len
    assert len(series) >= L, "Series too short!"

    # Select the last L values
    data = series[-L:]
    # Select the last L timestamps
    time_index = timestamps[-L:]

    # Initialize scaler and normalize the data
    scaler = StandardScaler()
    data = scaler.fit_transform(data)


    if features == 'S':
        # Univariate input/output
        if data.ndim == 2:
            x = data[:seq_len, target_index]
            y = data[seq_len - label_len:, target_index]
        else:
            x = data[:seq_len]
            y = data[seq_len - label_len:]
        batch_x = x.reshape(1, seq_len, 1)
        batch_y = y.reshape(1, label_len + pred_len, 1)
    elif features == 'M':
        # Multivariate input/output
        if data.ndim == 1:
            data = data[:, None]
        x = data[:seq_len, :]
        y = data[seq_len - label_len:, :]
        batch_x = x.reshape(1, seq_len, -1)
        batch_y = y.reshape(1, label_len + pred_len, -1)
    elif features == 'MS':
        # Multivariate input, univariate output
        if data.ndim == 1:
            raise ValueError("For 'MS', input must be multivariate (2D array)")
        x = data[:seq_len, :]
        y = data[seq_len - label_len:, target_index]
        batch_x = x.reshape(1, seq_len, -1)
        batch_y = y.reshape(1, label_len + pred_len, 1)
    else:
        raise ValueError("features must be 'S', 'M', or 'MS'")

    # Prepare time features (using pandas and your codebase's timefeatures)
    x_mark = time_index[:seq_len]
    y_mark = time_index[seq_len - label_len:]

    # Convert to pandas DatetimeIndex before passing to time_features_fixed
    batch_x_mark = time_features_fixed(pd.DatetimeIndex(x_mark)).reshape(1, seq_len, -1)
    batch_y_mark = time_features_fixed(pd.DatetimeIndex(y_mark)).reshape(1, label_len + pred_len, -1)

    # Prepare dec_inp: first label_len steps from batch_y, then pred_len zeros
    dec_zeros = np.zeros_like(batch_y[:, -pred_len:, :])
    dec_inp = np.concatenate([batch_y[:, :label_len, :], dec_zeros], axis=1)

     # Convert numpy arrays to torch tensors
    batch_x = torch.from_numpy(batch_x).float()
    batch_y = torch.from_numpy(batch_y).float()
    batch_x_mark = torch.from_numpy(batch_x_mark).float()
    batch_y_mark = torch.from_numpy(batch_y_mark).float()
    dec_inp = torch.from_numpy(dec_inp).float()


    return batch_x, batch_y, batch_x_mark, batch_y_mark, dec_inp, scaler # Return scaler

In [ ]:
series_values = df_etth1.drop(columns=['date']).values # Drop the 'date' column
timestamps = pd.to_datetime(df_etth1['date']) # Get the timestamps as datetime objects

batch_x, batch_y, batch_x_mark, batch_y_mark, dec_inp, scaler = prepare_eval_sample(
    series_values,
    timestamps, # Pass timestamps
    features='M'
)

In [ ]:
batch_x.shape, batch_y.shape, batch_x_mark.shape, batch_y_mark.shape, dec_inp.shape

In [ ]:
with torch.no_grad():
  preds = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

In [ ]:
preds_feature0 = preds[0, :, 0]
truth_feature0 = batch_y[0, -96:, 0]
plt.figure(figsize=(20, 6))
plt.plot(preds_feature0, label='pred')
plt.plot(truth_feature0, label='true')
plt.legend()
plt.show()

In [ ]:
series_values = df_etth1.drop(columns=['date']).values
display(series_values)

In [ ]:
series_values.shape